In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
%%capture
!pip install ftfy translate langdetect imbalanced-learn collections-extended rake_nltk googletrans

In [9]:
import sys
sys.path.append('/content/drive/My Drive/Capstone')

In [10]:
# !pip install chart_studio
# !pip install wordcloud
# !pip install ftfy
# !pip install rake_nltk
# !pip install gensim 
# !pip install missingno
# !pip install googletrans
# !pip install langdetect
# !pip install imblearn
# !pip install collections2

# Standard Library
import pandas as pd
import numpy as np

from ftfy import *
import nltk
from rake_nltk import Rake

from utils.visualization import *
from utils.datapreprocessing import *

import missingno as msno
import matplotlib.pyplot as plt 
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns 

# For Translation
from googletrans import Translator
from langdetect import detect

#For Upsampling
import imblearn
from collections import Counter
from matplotlib import pyplot
from numpy import where
# from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

# Import label encoder 
from sklearn import preprocessing 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer

# Import Dependencies

import os
import re

import tensorflow as tf
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

import keras

from keras import layers
# For Embedding Layer
from keras.layers.embeddings import Embedding

# For RNN Layer
from keras.models import Sequential
from keras.layers import Embedding, GRU, LSTM, Bidirectional
from keras.layers import Dense, Dropout, Activation, Input,  Flatten

# import numpy as np
# from tensorflow import keras
# from tensorflow.keras import layers

# For CNN Layer
from keras.layers import Conv1D, MaxPooling1D

# For Text Summarization
from gensim.summarization import summarize

%tensorflow_version 1.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [25]:
# Importing Final Dataset after upsampling and clustering

In [11]:

filename= "/content/drive/My Drive/Capstone/final_dataframe_forDeep_Learning_Model.csv"
df = pd.read_csv(filename)

In [26]:
# Label Encoding of Target variable

In [12]:
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['Assignment group'] = label_encoder.fit_transform(df['Assignment group']) 
  
df['Assignment group'].unique() 

array([ 8, 30, 41, 52, 63, 74, 79, 80,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 34, 35, 36,
       37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 62, 64, 65, 33, 66, 67, 68, 69, 70, 71, 72,
       73, 75, 76, 77, 78,  2,  3,  0,  1,  4,  6,  5,  7])

In [28]:
# Tokenizing and Padding the Independent features Complete Description

In [13]:
MAX_LENGTH = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.Complete_Description.values)
post_seq = tokenizer.texts_to_sequences(df.Complete_Description.values)
post_seq_padded = pad_sequences(post_seq, maxlen=MAX_LENGTH)

In [14]:
y = df['Assignment group']

In [29]:
# Splitting the data into Train and Test (Ratio = 0.30)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(post_seq_padded, y, test_size=0.3, random_state=1)

In [16]:
X_train.shape,X_test.shape

((36560, 300), (15669, 300))

In [17]:
y_train.shape, y_test.shape

((36560,), (15669,))

In [30]:
# Creating input dimension ('vocab_size')

In [18]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

16390

In [19]:
df.info

<bound method DataFrame.info of                                     Complete_Description  Assignment group
0      event: critical:HostName_221.company.com the v...                 8
1      when undocking pc , screen will not come back ...                30
2        received from: emailaddress  gentles,  i hav...                41
3        received from: emailaddress  hi -  the print...                52
4      received from: emailaddress  job Job_1424 fail...                63
...                                                  ...               ...
52224  name:mfeyouli ndobtzpw language: browser:micro...                 2
52225                      account locked account locked                 1
52226  hr_tool etime option not visitble  hr_tool eti...                 6
52227  telephony_software issue telephony_software issue                 2
52228  vip2: windows password reset for tifpdchb pedx...                 2

[52229 rows x 2 columns]>

In [31]:
# Creating Output dimension ('num_class')

In [20]:
num_class = len(np.unique(df['Assignment group'].values))
num_class

81

## Bidirectional LSTM

In [21]:
max_features = vocab_size  
maxlen = 200
seq_output_size = 128
filters = 64
kernel_size = 5
pool_size = 4
activation_func = 'sigmoid'

In [22]:
# Bi directional LSTM
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128)(inputs)
x = layers.Dropout(0.2)(x)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation='relu')(x)
outputs = layers.Dense(num_class, activation='softmax')(x)
# Add a classifier
bilstm_model = keras.Model(inputs, outputs)
bilstm_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 128)         2097920   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 128)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 128)         98816     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               98816     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4128

In [23]:
#model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
bilstm_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
bilstm_model.fit(X_train, y_train, batch_size=20, epochs=7, validation_data=(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 36560 samples, validate on 15669 samples
Epoch 1/7
36560/36560 [==============================] - 855s 23ms/step - loss: 1.6869 - acc: 0.5708 - val_loss: 0.7063 - val_acc: 0.8193
Epoch 2/7
36560/36560 [==============================] - 862s 24ms/step - loss: 0.5394 - acc: 0.8552 - val_loss: 0.4316 - val_acc: 0.8914
Epoch 3/7
36560/36560 [==============================] - 862s 24ms/step - loss: 0.3404 - acc: 0.9085 - val_loss: 0.3400 - val_acc: 0.9161
Epoch 4/7
36560/36560 [==============================] - 864s 24ms/step - loss: 0.2434 - acc: 0.9344 - val_loss: 0.3003 - val_acc: 0.9245
Epoch 5/7
36560/36560 [==============================] - 864s 24ms/step - loss: 0.1942 - acc: 0.9474 - val_loss: 0.2924 - val_acc: 0.9324
Epoch 6/7
36560/36560 [==============================] - 861s 24ms/step - loss: 0.1591 - acc: 0.9570 - val_loss: 0.2702 - val_acc: 0.9404
Epoch 7/7
36560/36560 [==============================] - 863s 24ms/step - loss: 0.1211 - acc: 0.9669 - val_loss: 0.2618 - 

In [24]:
bilstm_predicted = bilstm_model.predict(X_test)
bilstm_predicted = np.argmax(bilstm_predicted, axis=1)
accuracy_score(y_test, bilstm_predicted)

0.9456251196630289